# 일반화 성능 : 한번 예측한 결과는 믿을만 한가?
---------------



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 1.Data 가져와서 둘러보기

In [ ]:
# 데이터를 불러옵시다.
iris = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/iris.csv")

In [ ]:
# 데이터를 살펴봅시다.
iris.head()

In [ ]:
iris.describe()

## 2.Random Sampling

* 데이터를 랜덤에서 분할하여 모델링 & 예측을 수행합니다.
* iris는 feature들이 모두 숫자이므로 dummy variable은 필요하지 않음
* Scaling만 필요


### 2.1.반복 실행해 봅시다.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree

* 아래 셀은 데이터 분할 > 스케일링 > 모델링 코드를 모두 모아놓았습니다.
* Ctrl+Enter로 반복 실행해 봅시다. 

In [ ]:
# 데이터 분할
target = 'Species'
x = iris.drop(target, axis = 1)
y = iris.loc[:, target]

train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=0.3)

# DecisionTree 모델링
model = DecisionTreeClassifier(max_depth = 3)
model.fit(train_x, train_y)
model_pred_val = model.predict(val_x)
accuracy_score(val_y, model_pred_val)

정분류율은 몇 %인가요?  
값이 달라지나요? 값이 달라지는 원인은 무엇인가요?   
값이 달라진다면 모델의 성능을 어떻게 믿을 수 있을까요?  

### 2.2.여러번 반복해서 평균을 계산해 봅시다.

In [ ]:
# 20번 반복 실행하여 결과를 담아 냅시다.

result = []
for i in range(1,21):
    train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=0.3)

    model = DecisionTreeClassifier(max_depth = 3)
    model.fit(train_x, train_y)
    model_pred_val = model.predict(val_x)
    result.append(accuracy_score(val_y, model_pred_val))

평균과 표준편차를 구해 봅시다.

In [ ]:
np.mean(result), np.std(result)

In [ ]:
np.mean(result) - 2* np.std(result), np.mean(result) + 2* np.std(result)

값의 분포를 살짝 살펴보면

In [ ]:
# matplotlib histogram
plt.boxplot(result)
plt.show()

#### (옵션) max_depth 값을 달리하며 20회씩 수행하고 결과를 비교해 봅시다.

In [ ]:
# 20번 반복 실행하여 결과를 담아 냅시다.
acc = []
m_depth = []
for d in [1,2,3,4] :
    for i in range(1,21):
        train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=0.3)

        model = DecisionTreeClassifier(max_depth = d)
        model.fit(train_x, train_y)
        model_pred_val = model.predict(val_x)
        acc.append(accuracy_score(val_y, model_pred_val))
        m_depth.append(d)

In [ ]:
result1 = pd.DataFrame({'maxDepth': m_depth, 'Acc':acc})
result1.boxplot('Acc', 'maxDepth')
plt.show()

### 실습 : knn 알고리즘으로 실습해 봅시다.

1. k를 지정하지 말고(default)로 50회 수행한 후 결과를 담고 평균으로 일반화 성능을 얻어 봅시다.
2. k를 3,5,7,9 로 조정하며, 각 50회씩 수행한 후 결과를 담고, 각 분포로 성능을 비교해 봅시다.


## 3.k-fold Cross Validation

계획적으로 분할, 학습, 검증하는 방식을 사용해 봅시다!


In [ ]:
# 다시 데이터 가져와서
iris = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/iris.csv")

# features와 target 분리
target = 'Species'
x = iris.drop(target, axis = 1)
y = iris.loc[:, target]

# train+val : test = 8 : 2
train_val_x, test_x, train_val_y, test_y = train_test_split(x, y, test_size=0.2)

* cross validation 함수를 이용하여 모델을 검증합니다.

In [ ]:
# 필요햔 패키지, 함수 로딩
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# 모델 선언 
model = DecisionTreeClassifier(max_depth = 3)

# train + validation set을 이용하여 학습, 예측, 평가를 한번에. (여기서는 .fit 이 아님!)
cv_result = cross_val_score(model, train_val_x, train_val_y, cv=10)
print(cv_result)
print(cv_result.mean())

In [ ]:
# 역시 max_depth 값을 조정해 가며 성능을 비교해 봅시다.
# 100번 반복 실행하여 결과를 담아 냅시다.
scores = []
m_depth = []
for d in [1,2,3,4] :
    model = DecisionTreeClassifier(max_depth = d)
    cv_result = cross_val_score(model, train_val_x, train_val_y, cv=10)
    scores = scores + cv_result.tolist()
    m_depth = m_depth + [d]*10

In [ ]:
result2 = pd.DataFrame({'maxDepth': m_depth, 'Acc':scores})
result2.boxplot('Acc', 'maxDepth')
plt.show()

* 반복실행한 결과와 cv의 결과를 평균으로 비교해 봅시다.
* (주의) 데이터 사이즈(건수)가 많을 수록 둘의 편차는 줄어들 수 있습니다.

In [ ]:
# result1
result1.groupby('maxDepth', as_index=False)['Acc'].mean()

In [ ]:
result2.groupby('maxDepth', as_index=False)['Acc'].mean()

우리는 여기서 나온 평균 성능을, 모델의 일반화 성능으로 이야기 할 수 있습니다~!

## 실습 : knn알고리즘을 이용하여 k-fold cv 수행
1. fold의 갯수 5, 
2. 이웃의 갯수(k 값) : 3,5,7,9 에 대해서
3. cv score 평균을 가지고 비교해 봅시다. 

## 4.(Optional)학습 데이터 양에 따른 성능(Learning Curve)

학습할 데이터를 증가시켜가면서 성능을 확인해보고자 합니다.

In [ ]:
from sklearn.model_selection import learning_curve

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=2)

In [ ]:
x.shape[0]

In [ ]:
tr_size, tr_scores, val_scores = learning_curve(model, x, y
                                                , train_sizes = range(1, 120)
                                                , cv = 5)

In [ ]:
# cv한 결과를 평균으로 집계 합시다.
tr_scores_mean = tr_scores.mean(axis = 1)
val_scores_mean = val_scores.mean(axis = 1)

In [ ]:
# 이제 그림을 그려봅시다.
plt.plot(tr_size, val_scores_mean)
plt.show()

In [ ]:
# 어느 정도 데이터이면 학습하는데 충분할까요?




### 실습

In [ ]:
from sklearn.impute import SimpleImputer,KNNImputer

In [ ]:
# mobile data로 적정 학습 데이터 크기를 찾아 봅시다.
data_path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_NA2.csv"
mobile = pd.read_csv(data_path)

In [ ]:
# 데이터 전처리
target = 'CHURN'
x = mobile.drop(target, axis=1)
y = mobile.loc[:, target]

# 변수 삭제
col_drop = ['id', 'COLLEGE']
x.drop(col_drop, axis = 1, inplace=True)

# REPORTED_SATISFACTION 채우기
s_imputer = SimpleImputer(strategy = 'constant', fill_value = 'avg')
x[['REPORTED_SATISFACTION']] = s_imputer.fit_transform(x[['REPORTED_SATISFACTION']])

# 가변수화
col_cat = ['REPORTED_SATISFACTION','REPORTED_USAGE_LEVEL','CONSIDERING_CHANGE_OF_PLAN']
for v in col_cat : 
    dumm = pd.get_dummies(x[v], prefix= v)
    x = pd.concat([x, dumm], axis=1)
    x.drop(v, axis=1, inplace=True)

col_x = list(x)
house_knnimp = KNNImputer()
x = house_knnimp.fit_transform(x)
x = pd.DataFrame(x, columns=col_x)


여기서 부터 시작입니다.


In [ ]:
# 간단한 Decision Tree 모델을 선언합니다.
# max_depth는 3~5 사이로 결정



In [ ]:
# learning curve를 수행합니다.



In [ ]:
# plotting 하여 적절한 데이터 사이즈를 찾아 봅시다.

